<a href="https://colab.research.google.com/github/ohashi-gnct/exp/blob/2022/signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ライブラリのインポート
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

import librosa
import librosa.display
import soundfile as sf
from IPython.display import display, Audio

sr = 44100

Googleドライブ直下に以下のファイル名でファイルを置く。

- `source.wav`: 無響音源
- `ir1.wav`: 測定した1つ目の部屋でのインパルス応答
- `ir2.wav`: 測定した2つ目の部屋でのインパルス応答
- `recordedresponse1.wav`: 無響音源を1つ目の部屋で流して録音したもの
- `recordedresponse2.wav`: 無響音源を2つ目の部屋で流して録音したもの


これらのファイルをColabで読み込めるようにする。

In [ ]:
# Googleドライブの接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_path = '/content/drive/MyDrive'

# 3.2 たたみ込みによる音響再現

**※3.3の結果が3.2を包含しているため、3.2の実験結果は記述しなくてよい。**

まず、波形を表示する関数を用意する。

In [ ]:
# 波形と音源を表示するための関数
def show_wave_audio(y, sr):
  plt.figure(figsize=(10, 4))
  librosa.display.waveplot(y, sr=sr, x_axis='time')
  plt.show()
  display(Audio(y, rate=sr))

無響音源を読み込んで表示する。

In [ ]:
# 無響音源の読み込み
# ドライブ直下にファイルがある場合、下のように読み込める
file_source = "source.wav"
source, _ = librosa.load(os.path.join(drive_path, file_source), sr=sr)
show_wave_audio(source, sr)

インパルス応答1を読み込んで表示する。

In [ ]:
# インパルス応答の読み込み
file_ir1 = "ir1.wav"
ir1, _ = librosa.load(os.path.join(drive_path, file_ir1), sr=sr)
show_wave_audio(ir1, sr)

無響音源とインパルス応答を受け取って、たたみ込みを行った結果を返す関数を作成する。

In [ ]:
# たたみ込みを行う関数
def calc_response(source, ir):
  # TODO: たたみ込みのプログラムを考える
  response = 

  # 音を再生するため、サンプルの最大値を1にする
  response /= np.amax(response)
  return(response)

無響音源とインパルス応答1のたたみ込みを行う。

In [ ]:
# たたみ込み
show_wave_audio(calc_response(source, ir1), sr)

たたみ込みは`scipy.signal.fftconvolve()`でも行うことができる。

愚直なたたみ込みより、FFTを使ったたたみ込みのほうが速い。

In [ ]:
# FFTを使ったたたみ込み
def calc_response_fft(source, ir):
  response = signal.fftconvolve(source, ir)
  response /= np.amax(response)
  return(response)

たたみ込みの結果は同じになる。

In [ ]:
response1 = calc_response_fft(source, ir1)
show_wave_audio(response1, sr)

この結果を録音したものと比較する。

In [ ]:
file_recordedresponse1 = "recordedresponse1.wav"
recordedresponse1, _ = librosa.load(os.path.join(drive_path, 
                                                 file_recordedresponse1), sr=sr)
show_wave_audio(recordedresponse1, sr)

# 3.3 音声の周波数解析

まず、波形、スペクトログラムを同時に表示する関数を用意する。

In [ ]:
# 波形とスペクトログラムと音源を表示するための関数
def show_wave_spectrogram_audio(y, sr):
  fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(12, 10))
  S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
  log_S = librosa.power_to_db(S, ref=np.max)
  librosa.display.waveplot(y, sr=sr, x_axis='time', ax=ax1)
  librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel', ax=ax2)
  plt.show()
  display(Audio(y, rate=sr))

無響音源の波形とスペクトログラム

In [ ]:
show_wave_spectrogram_audio(source, sr)

インパルス応答1の波形とスペクトログラム

In [ ]:
show_wave_spectrogram_audio(ir1, sr)

無響音源にインパルス応答1を畳み込んだ結果の波形とスペクトログラム

In [ ]:
show_wave_spectrogram_audio(response1, sr)

無響音源を部屋1で流して録音した波形とスペクトログラム

In [ ]:
show_wave_spectrogram_audio(recordedresponse1, sr)

インパルス応答2についても同様に畳み込む。

インパルス応答2の波形とスペクトログラム

In [ ]:
# インパルス応答の読み込み
file_ir2 = "ir2.wav"
ir2, _ = librosa.load(os.path.join(drive_path, file_ir2), sr=sr)

show_wave_spectrogram_audio(ir2, sr)

無響音源にインパルス応答2を畳み込んだ結果の波形とスペクトログラム

In [ ]:
response2 = calc_response_fft(source, ir1)
show_wave_spectrogram_audio(response2, sr)

無響音源を部屋2流して録音した波形とスペクトログラム

In [ ]:
# インパルス応答の読み込み
file_recordedresponse2 = "recordedresponse2.wav"
recordedresponse2, _ = librosa.load(os.path.join(drive_path, 
                                                 file_recordedresponse2), sr=sr)
show_wave_spectrogram_audio(recordedresponse2, sr)

# おまけ

極端に高周波が響かないインパルス応答を人工的に作ってみる

ここではsinc関数を作る。

$$
\rm{sinc}(x) = \frac{\sin{(x)}}{x}$$

In [ ]:
x = np.linspace(0.001, 100, 100000)
sinc = np.sin(x) / x

In [ ]:
show_wave_spectrogram_audio(sinc, sr)

これを無響音源にたたみ込むと…。

In [ ]:
show_wave_spectrogram_audio(calc_response_fft(source, sinc), sr)